<a href="https://colab.research.google.com/github/MichalWlodkowski/US_speeches/blob/master/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from scrapy import Selector
import requests
import numpy as np
import pandas as pd
import re

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize

In [0]:
us_all = 'https://en.wikisource.org/wiki/Portal:Inaugural_Speeches_by_United_States_Presidents'

In [0]:
## Getting links to all presidential Inaugural Addresses in US

html = requests.get(us_all).content
sel = Selector( text = html )
xpath3 = '//@href'
d = pd.DataFrame(sel.xpath(xpath3).extract())
speech = d[d[0].str.contains('Inaugural_Address')].reset_index(drop = True)
speech.columns = ['links']

In [6]:
# Function to get president name from links + receiving full link for speeches
def frag(x):
    wh = x[6:x.find('%')].replace('_',' ')
    return wh
speech['president'] = speech['links'].apply(frag)
speech['full_links'] = speech['links'].apply(lambda x: 'https://en.wikisource.org' + x)
speech.head()

,links,president
0,/wiki/George_Washington%27s_First_Inaugural_Ad...,George Washington
1,/wiki/George_Washington%27s_Second_Inaugural_A...,George Washington
2,/wiki/John_Adams%27_Inaugural_Address,John Adams
3,/wiki/Thomas_Jefferson%27s_First_Inaugural_Add...,Thomas Jefferson
4,/wiki/Thomas_Jefferson%27s_Second_Inaugural_Ad...,Thomas Jefferson


In [0]:
plain_text = []
#for each in range(len(speech)-3,len(speech)):
for each in range(len(speech)):
    
    html = requests.get(speech.full_links.loc[each]).content
    sel = Selector (text = html)
    xpath3 = '//*[@class="mw-parser-output"]//p//text()'
    # Extract text from website + join in 1string + remove unwanted text and new lines
    text = ''.join(sel.xpath(xpath3).extract()).replace('\nThis work is in the public domain in the United States because it is a work of the United States federal government (see 17\xa0U.S.C.\xa0105).\n','')\
.replace('\n',' ')
    plain_text.append(text)    

In [0]:
def hm_words(x):
    words = len(x.split())
    return words

def hm_ch_words(x):
    words_ch = round(np.mean([len(x) for x in x.split()]),2)
    return words_ch

def n_sentences(x):
    sent = len(x.split('.'))
    return sent

def n_parag(x):
    par =  len(x.split('\n'))
    return par

In [27]:
speeches = pd.DataFrame(plain_text, columns = ['speeches'])
speeches['line_text'] = pd.DataFrame(text_lines)
speeches['characters'] = speeches.speeches.apply(len)
speeches['words'] = speeches.speeches.apply(hm_words)
speeches['ch_in_words'] = speeches.speeches.apply(hm_ch_words)
speeches['sentences'] = speeches.speeches.apply(n_sentences)
speeches['paragraphs'] = speeches.line_text.apply(n_parag)

speeches['ch_in_sent'] = round(speeches['characters'] / speeches['sentences'],1)
speeches.tail(2)

,speeches,line_text,characters,words,ch_in_words,sentences,paragraphs,ch_in_sent
56,"Vice President Biden, Mr. Chief Justice, Memb...","\nVice President Biden, Mr. Chief Justice, Mem...",11954,2114,4.65,86,23,139.0
57,"Chief Justice Roberts, President Carter, Presi...","Chief Justice Roberts, President Carter, Presi...",8413,1456,4.78,96,16,87.6


In [28]:
db = pd.concat([speech['president'],speeches.iloc[:,2:]], axis = 1)
db.tail()

,president,characters,words,ch_in_words,sentences,paragraphs,ch_in_sent
53,George W. Bush,9005,1586,4.68,84,20,107.2
54,George W. Bush,11918,2083,4.72,104,181,114.6
55,Barack Obama,13140,2328,4.64,109,35,120.6
56,Barack Obama,11954,2114,4.65,86,23,139.0
57,Donald Trump,8413,1456,4.78,96,16,87.6


In [29]:
round(db.describe().T,1)

,count,mean,std,min,25%,50%,75%,max
characters,58.0,13681.4,8321.8,786.0,8114.2,11936.0,17489.0,49652.0
words,58.0,2333.4,1387.1,135.0,1444.0,2093.5,2886.8,8424.0
ch_in_words,58.0,4.8,0.2,4.4,4.7,4.9,5.0,5.1
sentences,58.0,87.0,47.0,5.0,43.5,89.0,115.8,213.0
paragraphs,58.0,38.1,40.1,3.0,15.2,27.0,40.8,210.0
ch_in_sent,58.0,171.7,70.4,87.2,117.8,152.1,206.8,407.7


In [30]:
db.groupby(['president']).mean().sort_values('characters', ascending = False).head()

,characters,words,ch_in_words,sentences,paragraphs,ch_in_sent
president,,,,,,
William Henry Harrison,49652.0,8424.0,4.89,213.0,25.0,233.1
William Howard Taft,32119.0,5428.0,4.92,161.0,44.0,199.5
James K. Polk,28695.0,4793.0,4.98,147.0,31.0,195.2
Benjamin Harrison,26195.0,4392.0,4.96,153.0,36.0,171.2
Calvin Coolidge,23946.0,4056.0,4.90,197.0,46.0,121.6
